<a href="https://colab.research.google.com/github/srush/drop7/blob/main/Drop7_penzai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
try:
  import penzai
except ImportError:
  !pip install penzai[notebook]

In [ ]:
import jax.numpy as np
import jax
from jax.lax import scan
import random
from IPython.display import display, clear_output
import numpy as onp
import penzai
from penzai import pz
pz.ts.register_as_default()

# Optional automatic array visualization extras:
pz.ts.register_autovisualize_magic()
pz.enable_interactive_context()
pz.ts.active_autovisualizer.set_interactive(pz.ts.ArrayAutovisualizer())
init = """
_____3_
2_3____
4_24477
4155762
4466256
XXXXXXX
XXXXXXX
""".strip().split()
board = onp.zeros((7, 7, 7), int)
block = onp.zeros((7, 7), int)
for r in range(7):
    for c in range(7):
        sym = init[r][c]
        if sym != "_":
            if sym == "X":
                board[6 - r, c, random.randint(0, 6)] = 1
                block[6-r, c] = 1
            else:
                board[6 - r, c, int(sym) - 1] = 1
cols = pz.nx.arange("col", 7)
rows = pz.nx.arange("row", 7)
vals = pz.nx.arange("val", 7)
nmap = pz.nx.nmap
nsum = lambda n: lambda x: nmap(np.sum)(x.untag(n))
nop = lambda f, *n: lambda x, *extra: nmap(f)(x.untag(*n), *extra).tag(*n)

board = pz.nx.wrap(board).tag("row", "col", "val")
block = pz.nx.wrap(block).tag("row", "col")
board


<NamedArray int32(| row:7, col:7, val:7) [≥0, ≤1] zero:306 nonzero:37 (wrapping jax.Array)
  <Arrayviz rendering>
>

## Drawing the board.

In [ ]:
def draw(board, block, burst=np.zeros((7, 7))):
    out = nsum("val")(board * (vals + 1)) * (1-block) - block
    out = nop(np.flip, "row")(out)
    return out
pz.ts.render_array(draw(board, block), columns=["col"], rows=["row"])

ArrayvizRendering(html_src='<div id="arrayvizd5efdd61ac344e7484e077cc6b5f17b4" class="arrayviz_container"><span class="loading_message">Rendering array...</span></div><template class="treescope_run_soon"><script> arrayviz.buildArrayvizFigure({"destinationId": "arrayvizd5efdd61ac344e7484e077cc6b5f17b4", "info": "Showing integer digits as nested squares. Hover/click for array data.", "arrayBase64": "AAAAAAIAAAAEAAAABAAAAAQAAAD//////////wAAAAAAAAAAAAAAAAEAAAAEAAAA//////////8AAAAAAwAAAAIAAAAFAAAABgAAAP//////////AAAAAAAAAAAEAAAABQAAAAYAAAD//////////wAAAAAAAAAABAAAAAcAAAACAAAA//////////8DAAAAAAAAAAcAAAAGAAAABQAAAP//////////AAAAAAAAAAAHAAAAAgAAAAYAAAD//////////w==", "arrayDtype": "int32", "validMaskBase64": "AQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQ==", "dataStrides": {"a1": 1, "a0": 7}, "xAxisSpecs": [{"name": "a0", "label": "col: 7", "start": 0, "end": 7}], "yAxisSpecs": [{"name": "a1", "label": "row: 7", "start": 0, "end": 7}], "slicedAxisSpecs": [], "colormapConfig": {"type": "digitbox", "min": null, "max": null, "dynamic": false, "rawMinAbs": null, "rawMaxAbs": null, "cmapData": []}, "valueFormattingInstructions": [{"type": "literal", "value": "array"}, {"type": "literal", "value": "[{"}, {"type": "literal", "value": "\'col\':"}, {"type": "index", "axis": "a0", "skip_start": 7, "skip_end": 7}, {"type": "literal", "value": ", \'row\':"}, {"type": "index", "axis": "a1", "skip_start": 7, "skip_end": 7}, {"type": "literal", "value": "}]"}, {"type": "literal", "value": "\\n  = "}, {"type": "value"}]})</script></template>')

In [ ]:
def fall(b):
    up = nop(np.convolve, "row")(nsum("val")(b) * 2-1,
                                 np.array([0, 1, -1]), "same") >= 2
    return b * (1-up) + nop(np.roll, "row")(b * up, -1, 0), up

def reduce(b, c, n):
    def count(arr, rev=False):
        return scan(lambda c, x: 2*(x * c + x,), 0, arr, reverse=rev)[1]

    bs = nsum("val")(b * (vals + 1))
    exp = (c == 0) & (bs == (nop(count, n)(bs > 0) + nop(count, n)(bs > 0, True) - 1))
    return b * (1-exp), exp

def unblock(exp, n):
    return nop(np.convolve, n)(exp, np.array([1, 0, 1]), "same")


In [ ]:
def run(board, block):
    boards = [(board, block)]
    while True:
        board, keep = fall(board)
        boards.append((board, block))
        if not keep.untag("col", "row").unwrap().any():
            board1, exp1 = reduce(board, block, "col")
            board2, exp2 = reduce(board, block, "row")
            exp = exp1 + exp2
            board = board1 & board2
            if exp.untag("col", "row").unwrap().sum() != 0:
                for i in range(2):
                    boards.append((board, block, exp))
                    boards.append((board, block, ))
            block = nmap(np.maximum)(pz.nx.wrap(0),
                                     block - (unblock(exp, "col") + unblock(exp, "row")))
            boards.append((board, block,))
            if exp.untag("col", "row").sum() == 0:
                break
    return boards

output = run(board, block)

out = pz.nx.stack([draw(o[0], o[1]) for o in (output)],
            "time")
out = nmap(np.int32)(out)
pz.ts.render_array(out, columns=["col"], rows=["row"], sliders=["time"], continuous=False)

ArrayvizRendering(html_src='<div id="arrayviz142cc2bf4cda4b0bb2830b1deff9a93e" class="arrayviz_container"><span class="loading_message">Rendering array...</span></div><template class="treescope_run_soon"><script> arrayviz.buildArrayvizFigure({"destinationId": "arrayviz142cc2bf4cda4b0bb2830b1deff9a93e", "info": "Showing integer digits as nested squares. Hover/click for array data.", "arrayBase64": "AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAACAAAAAgAAAAIAAAACAAAAAgAAAAIAAAACAAAAAgAAAAIAAAACAAAAAgAAAAIAAAACAAAAAgAAAAIAAAACAAAAAgAAAAIAAAACAAAAAgAAAAIAAAACAAAAAgAAAAIAAAACAAAAAgAAAAIAAAACAAAAAgAAAAIAAAACAAAAAgAAAAIAAAACAAAAAgAAAAIAAAACAAAAAgAAAAIAAAACAAAAAgAAAAIAAAACAAAAAgAAAAIAAAACAAAAAgAAAAIAAAACAAAAAgAAAAQAAAAEAAAABAAAAAQAAAAEAAAABAAAAAQAAAAEAAAABAAAAAQAAAAEAAAABAAAAAQAAAAEAAAABAAAAAQAAAAEAAAABAAAAAQAAAAEAAAABAAAAAQAAAAEAAAABAAAAAQAAAAEAAAABAAAAAQAAAAEAAAABAAAAAQAAAAEAAAABAAAAAQAAAAEAAAABAAAAAQAAAAEAAAABAAAAAQAAAAEAAAABAAAAAQAAAAEAAAABAAAAAQAAAAEAAAABAAAAAQAAAAEAAAABAAAAAQAAAAEAAAABAAAAAQAAAAEAAAABAAAAAQAAAAEAAAABAAAAAQAAAAEAAAABAAAAAQAAAAEAAAABAAAAAQAAAAEAAAABAAAAAQAAAAEAAAABAAAAAQAAAAEAAAABAAAAAQAAAAEAAAABAAAAAQAAAAEAAAABAAAAAQAAAAEAAAABAAAAAQAAAAEAAAABAAAAAQAAAAEAAAABAAAAAQAAAAEAAAABAAAAAQAAAAEAAAABAAAAAQAAAAEAAAABAAAAAQAAAAEAAAABAAAAAQAAAAEAAAABAAAAAQAAAAEAAAABAAAAAQAAAAEAAAABAAAAAQAAAAEAAAABAAAAAQAAAAEAAAABAAAAAQAAAAEAAAABAAAAAQAAAAEAAAABAAAAAQAAAAEAAAABAAAAAQAAAAEAAAABAAAAAQAAAAEAAAABAAAAAQAAAAEAAAABAAAAAQAAAAEAAAABAAAAAQAAAAEAAAABAAAAAQAAAAEAAAABAAAAAQAAAAEAAAABAAAAAQAAAAEAAAABAAAAP////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////8AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAABAAAAAQAAAAEAAAABAAAAAQAAAAEAAAABAAAAAQAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAQAAAAEAAAABAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAQAAAAEAAAABAAAAAQAAAAEAAAABAAAAAQAAAAEAAAABAAAAAQAAAAEAAAABAAAAAQAAAAEAAAABAAAAAQAAAAEAAAABAAAAAQAAAAEAAAABAAAAAQAAAAEAAAABAAAAAQAAAAEAAAABAAAAAQAAAAEAAAABAAAAAQAAAAEAAAABAAAAAQAAAAEAAAABAAAAAQAAAAEAAAABAAAAAQAAAAEAAAABAAAA/////////////////////////////////////wYAAAAGAAAABgAAAAYAAAAGAAAABgAAAAYAAAAGAAAABgAAAAYAAAAGAAAABgAAAAYAAAAGAAAABgAAAAYAAAAGAAAABgAAAAYAAAAGAAAABgAAAAYAAAAGAAAABgAAAAYAAAAGAAAABgAAAAYAAAAGAAAABgAAAAYAAAAGAAAABgAAAAYAAAAGAAAABgAAAAYAAAAGAAAABgAAAAYAAAAGAAAABgAAAAYAAAD////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////